# Grad-CAM Explainability

This notebook loads the trained EfficientNetB3 plant disease model and uses Grad-CAM to visualize **which regions of a plant image the model focuses on** when making a prediction.

You can:
- Pick a test image
- See prediction + Grad-CAM heatmap + overlay.


# 1. Imports


In [2]:
! pip install opencv-python
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print(cv2.__version__)


TensorFlow version: 2.20.0
Keras version: 3.12.0
4.12.0


# 2. Model path and class names


In [9]:
MODEL_PATH = "Models/EfficientNetB3/efficientnetb3_model.keras"

In [8]:
# Class names 
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot',
    'Pepper__bell___healthy',
    'Potato___Early_blight',
    'Potato___Late_blight',
    'Potato___healthy',
    'Tomato__Target_Spot',
    'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus',
    'Tomato_healthy'
]


In [7]:
print("Model path:", MODEL_PATH)
print("Model file exists:", os.path.exists(MODEL_PATH))
print("Num classes:", len(CLASS_NAMES))


Model path: Models/EfficientNetB3/efficientnetb3_model.keras
Model file exists: True
Num classes: 9


## 3. Load model


In [10]:
model = keras.models.load_model(MODEL_PATH)
print("Model loaded successfully!")
print("Model output shape:", model.output_shape)

# Print model structure for verification
print("\nModel layers:")
for i, layer in enumerate(model.layers):
    print(f"  {i}: {layer.name} - {type(layer).__name__}")
    if hasattr(layer, 'output_shape'):
        print(f"      Output shape: {layer.output_shape}")

# Verify base model exists
base_model_found = False
for layer in model.layers:
    if 'efficientnet' in layer.name.lower():
        base_model_found = True
        print(f"\n✓ Found base model: {layer.name}")
        print(f"  Base model output shape: {layer.output_shape}")
        break

if not base_model_found:
    print("\n Warning: Could not find EfficientNetB3 base model in the expected location.")


Model loaded successfully!
Model output shape: (None, 9)

Model layers:
  0: efficientnetb3 - Functional
      Output shape: (None, 7, 7, 1536)
  1: global_average_pooling2d - GlobalAveragePooling2D
  2: dropout - Dropout
  3: dense - Dense
  4: dropout_1 - Dropout
  5: dense_1 - Dense

✓ Found base model: efficientnetb3
  Base model output shape: (None, 7, 7, 1536)
